# Koopman with inputs and control

KIC

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(precision=4, suppress=True, linewidth=300)

## Example 1—Linear system with inputs

In [70]:
""" 
The random disturbances for the input are generated with zero 
mean and gaussian distributed with a variance of 0.01.
"""

class Lswi:
    def __init__(self):
        mu = 0.1
        ld = 1.5
        dt = 1.
        self.G = np.array([
            [mu, 0., 0.],
            [0., ld, dt],
            [0., 0., 0.],
        ])
    
    def propagate(self, y):
        return self.G @ y.reshape((3,1))

y = np.array([5., 2., 0.])
lsys = Lswi()
ylst = [y.tolist()]
for i in range(5):
    y = lsys.propagate(y) + np.array([0., 0., np.random.normal(0, 0.01, 1)[0]]).reshape((3,1))
    ylst.append(y.flatten().tolist())

Y = np.array(ylst[:-1]).T
Z = np.array(ylst[1:]).T
print(Y)
print(Z)

A = Z @ np.linalg.pinv(Y)
print(A)

[[ 5.      0.5     0.05    0.005   0.0005]
 [ 2.      3.      4.4833  6.7114 10.0675]
 [ 0.     -0.0167 -0.0136  0.0004 -0.0073]]
[[ 0.5     0.05    0.005   0.0005  0.0001]
 [ 3.      4.4833  6.7114 10.0675 15.0939]
 [-0.0167 -0.0136  0.0004 -0.0073  0.006 ]]
[[ 0.1     0.      0.    ]
 [ 0.      1.5     1.    ]
 [-0.0036  0.0003  0.3719]]


In [65]:
lsys2 = Lswi()
lsys2.G[2,1] = -1

y = np.array([5., 2., 0.])
ylst = [y.tolist()]
for i in range(5):
    # y = lsys2.propagate(y) + np.array([0., 0., np.random.normal(0, 0.01, 1)[0]]).reshape((3,1))
    y = lsys2.propagate(y)
    y[2] = -1 * y[1]
    ylst.append(y.flatten().tolist())

Y = np.array(ylst[:-1]).T
Z = np.array(ylst[1:]).T
print(Y)
print(Z)

A = Z @ np.linalg.pinv(Y)
print(A)

[[ 5.      0.5     0.05    0.005   0.0005]
 [ 2.      3.      1.5     0.75    0.375 ]
 [ 0.     -3.     -1.5    -0.75   -0.375 ]]
[[ 0.5     0.05    0.005   0.0005  0.0001]
 [ 3.      1.5     0.75    0.375   0.1875]
 [-3.     -1.5    -0.75   -0.375  -0.1875]]
[[ 0.1 -0.  -0. ]
 [-0.   1.5  1. ]
 [ 0.  -1.5 -1. ]]


In [72]:
lsys3 = Lswi()
lsys3.G[2,2] = 0.99

y = np.array([5., 2., 1.])
ylst = [y.tolist()]
for i in range(5):
    y = lsys3.propagate(y)
    ylst.append(y.flatten().tolist())

Y = np.array(ylst[:-1]).T
Z = np.array(ylst[1:]).T
print(Y)
print(Z)

A = Z @ np.linalg.pinv(Y)
print(A)

[[ 5.      0.5     0.05    0.005   0.0005]
 [ 2.      4.      6.99   11.4651 18.1679]
 [ 1.      0.99    0.9801  0.9703  0.9606]]
[[ 0.5     0.05    0.005   0.0005  0.0001]
 [ 4.      6.99   11.4651 18.1679 28.2125]
 [ 0.99    0.9801  0.9703  0.9606  0.951 ]]
[[ 0.1   0.    0.  ]
 [ 0.    1.5   1.  ]
 [-0.    0.    0.99]]


## Example 2—Nonlinear system with inputs

In [78]:
class NLSwI:
    def __init__(self):
        mu = 2.0
        ld = 0.5
        self.dt = 2.0
        self.G = np.array([
            [mu, 0.,  0.  ],
            [0., ld, -ld  ],
            [0., 0., 2.*mu],
        ])
        self.Gn = np.eye(3) + self.G * 0.01

    def propagate(self, y, u):
        yn = self.Gn @ y.reshape((3,1))
        yn[1] += self.dt * u
        return yn

nlsys = NLSwI()

y = np.array([5., 2., 25.])
ylst = [y.tolist()]
for i in range(15):
    y = nlsys.propagate(y, 0)
    y[2] = y[0] ** 2 
    ylst.append(y.flatten().tolist())

Y = np.array(ylst[:-1]).T
Z = np.array(ylst[1:]).T
print(Y)
print(Z)

A = Z @ np.linalg.pinv(Y)
print(A)
print(nlsys.Gn)

[[ 5.      5.1     5.202   5.306   5.4122  5.5204  5.6308  5.7434  5.8583  5.9755  6.095   6.2169  6.3412  6.468   6.5974]
 [ 2.      1.885   1.7644  1.6379  1.5053  1.3664  1.2208  1.0684  0.9088  0.7418  0.5669  0.384   0.1927 -0.0074 -0.2166]
 [25.     26.01   27.0608 28.1541 29.2915 30.4749 31.706  32.987  34.3196 35.7062 37.1487 38.6495 40.2109 41.8355 43.5256]]
[[ 5.1     5.202   5.306   5.4122  5.5204  5.6308  5.7434  5.8583  5.9755  6.095   6.2169  6.3412  6.468   6.5974  6.7293]
 [ 1.885   1.7644  1.6379  1.5053  1.3664  1.2208  1.0684  0.9088  0.7418  0.5669  0.384   0.1927 -0.0074 -0.2166 -0.4353]
 [26.01   27.0608 28.1541 29.2915 30.4749 31.706  32.987  34.3196 35.7062 37.1487 38.6495 40.2109 41.8355 43.5256 45.284 ]]
[[ 1.02   -0.     -0.    ]
 [ 0.      1.005  -0.005 ]
 [-0.      0.      1.0404]]
[[ 1.02   0.     0.   ]
 [ 0.     1.005 -0.005]
 [ 0.     0.     1.04 ]]
